# Sentence Generator for Nigerian Indigenous Languages

# Load Data from Git URL

In [1]:
!wget https://raw.githubusercontent.com/Niger-Volta-LTI/yoruba-text/master/Owe/owe.txt

--2022-08-23 20:03:57--  https://raw.githubusercontent.com/Niger-Volta-LTI/yoruba-text/master/Owe/owe.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180836 (177K) [text/plain]
Saving to: ‘owe.txt.2’

owe.txt.2           100%[===================>] 176.60K  --.-KB/s    in 0.009s  

2022-08-23 20:03:57 (20.1 MB/s) - ‘owe.txt.2’ saved [180836/180836]



### Import Neccessary Libraries and Load Data

In [2]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [4]:
with open('owe.txt') as file:
    # Load data in a list
    data = file.readlines()
# Join eaxh line of proverb together, convert to lowercase and split by '\n'
data = ''.join(data).lower().split('\n') 
len(data)

2702

In [5]:
# Check a subset of the data
data[:10]

['',
 'a di gàárì sílẹ̀ ewúrẹ́ ńyọjú; ẹrù ìran rẹ̀ ni?',
 'a kì í dàgbà má làáyà; ibi ayé bá báni là ńjẹ ẹ́.',
 'àgbà kán ṣe bẹ́ẹ̀ lógùn; yemaja ló gbé e lọ.',
 'ibi tí oyín gbé ńhó, tí àdó ńhó, ìfun ò dákẹ́ lásán.',
 'ìdí òwò ni òwòó gbé tà.',
 'igún ṣoore ó pá lórí, àkàlà-á ṣoore ó yọ gẹ̀gẹ̀; nítorí ọjọ́ mìíràn kẹni ó má ṣe oore bẹ́ẹ̀ mọ́.',
 'ìgbà ara là ḿbúra.',
 'igbá là ńpa, a kì í pa àwo.',
 'ìgbà òjò ńlọ, ìgbà ẹ̀rùn ńlọ, a ní ká dí isà eku kó le; ìgbà wo la óò tó wá peku náà?']

Notice the empty line at index 0. deleting that index from our list

In [6]:
del data[0]

In [7]:
len(data)

2701

In [8]:
data[:5]

['a di gàárì sílẹ̀ ewúrẹ́ ńyọjú; ẹrù ìran rẹ̀ ni?',
 'a kì í dàgbà má làáyà; ibi ayé bá báni là ńjẹ ẹ́.',
 'àgbà kán ṣe bẹ́ẹ̀ lógùn; yemaja ló gbé e lọ.',
 'ibi tí oyín gbé ńhó, tí àdó ńhó, ìfun ò dákẹ́ lásán.',
 'ìdí òwò ni òwòó gbé tà.']